# Building

If you have already built the executable as `./build/sdf-weighted-delaunay` feel free to skip this part.

Otherwise, set `build=True` to build. 

In [ ]:
build   = False
binpath = "./build/sdf-weighted-delaunay"

import os
if build:
    if not os.path.exists("./build"):
        os.mkdir("build")
    !cmake -B build -DCGAL_DIR=./cgal 
    !cmake --build build/ --parallel
    
if os.path.exists(binpath): 
    print("Executable Available")
else:
    print("Error: Executable not available")

# Data

In [ ]:
data_folder = "data"
if not os.path.exists("data"):
    os.mkdir("data")
    
mesh_paths     = {"armadillo" : "data/armadillo.obj"}
download_paths = {"armadillo" : "https://raw.githubusercontent.com/alecjacobson/common-3d-test-models/refs/heads/master/data/armadillo.obj"}
                  

In [ ]:
import os
# Download meshes if they're not available
mesh_name = "armadillo"

if not os.path.exists(mesh_paths[mesh_name]):
    import wget
    url = download_paths[mesh_name]
    filename = wget.download(url,out=mesh_paths[mesh_name])
    print()
    print(filename)
else:
    print("Found .obj file")

# Contouring

We reference and recreate some figures from
- [KA25] Kohlbrenner, M. and Alexa, M. (2025), Isosurface Extraction for Signed Distance Functions using Power Diagrams. Computer Graphics Forum, 44: e70037. https://doi.org/10.1111/cgf.70037

In [ ]:
out_folder = "./output"
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

## Regular Contouring $\mathbf{RC}$
Regular contouring ($\mathbf{RC}$ in [KA25]) contours the regular triangulation without refinement.

As an example, the top row of Fig. 7 [KA25] can be recreated as follows. 

In [ ]:
mesh_path = mesh_paths["armadillo"]
Ns = [21,51,71]
max_refinement = 0
delaunay = False
for N in Ns:
    !{binpath} {mesh_path} {N} {max_refinement} {1 if delaunay else 0} {out_folder}

In [ ]:
import numpy as np
import polyscope as ps
from matplotlib.colors import to_rgb

camera_views = {"armadillo" : (v_ := np.array([0.,0.,-1.])*2.8,-v_) } 

from gpytoolbox import read_mesh
ps.init()
ps.set_ground_plane_mode("none")

screenshots = {}
for N in Ns:
    V, F = read_mesh(os.path.join(out_folder, f"armadillo.obj_RegMTR_{N}_0.obj"))
    ps.remove_all_structures()
    ps.register_surface_mesh("arma21", V, F, color=to_rgb("tab:green"))
    ps.look_at(*camera_views["armadillo"])
    screenshots[N] = ps.screenshot_to_buffer()   
    
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1,3,figsize=(20,10))

for ni,(N,im) in enumerate(screenshots.items()):
    ax = axs[ni]
    ax.set_title(N,fontsize=20)
    ax.imshow(screenshots[N][200:-200,800:-800])
    ax.axis("off")
plt.show()